# Entrega 1 - Introducción al Procesamiento del Lenguaje Natural 2018 


Verifique que su entorno de Python 3 contiene todas las bibliotecas necesarias. Importe las bibliotecas nltk y sklearn y verifique que se importan sin errores.

In [1]:
import nltk
nltk.download('punkt')
import sklearn
import json
import os
import re
from pprint import pprint
from random import shuffle, seed

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Luchong\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Lectura de datos

Observe el corpus contenido en el directorio *restaurante-review-dataset* extraído de [1]. Cargue el contenido del corpus en una variable de nombre *corpus*. Utilice como estructura de datos una lista de pares (comentario, valor), donde comentario es una string con el comentario y valor corresponde a la string "POS" o "NEG" si el comentario es positivo o negativo, respectivamente. Despliegue la cantidad de comentarios positivos y negativos.
 
[1] Dubiau, L., & Ale, J. M. (2013). Análisis de Sentimientos sobre un Corpus en Español: Experimentación con un Caso de Estudio. In Proceedings of the 14th Argentine Symposium on Artificial Intelligence, ASAI (pp. 36-47).


In [2]:
path = "../dataset/restaurante-review-dataset/"
posPath = path + "pos/"
negPath = path + "neg/"
posFiles = os.listdir(posPath)
negFiles = os.listdir(negPath)
posJsons = [json.load(open(posPath + file)) for file in posFiles]
negJsons = [json.load(open(negPath + file)) for file in negFiles]
posCorpus = [(comentario,"POS") for lista in posJsons for comentario in lista]
negCorpus = [(comentario,"NEG") for lista in negJsons for comentario in lista]
corpus = posCorpus + negCorpus
print(str(len(posCorpus)) + " comentarios positivos y " + str(len(negCorpus)) + " comentarios negativos.")

34808 comentarios positivos y 17633 comentarios negativos.


Guarde los comentarios (sin importar el valor que tenga asignado) en una lista llamada *comentarios*. Reordene esta lista aleatoriamente. Utilice una semilla para el generador aleatorio para que sus resultados sean reproducibles. Despliegue los 3 primeros elementos de la lista *comentarios*.

In [3]:
comentarios = [comentario for (comentario,_) in corpus]
seed(3)
shuffle(comentarios)
pprint(comentarios[0:3])

['Algo diferente sobre todo por la cordialidad y el servicio. Es en una casa '
 'antigua, dentro de una biblioteca publica, la comida excelentemente '
 'casera,   y la atencion buenisima.',
 'soy un neófito en la cultura gastronómica de buen nivel y aún así el trato y '
 'la amabilidad son destacables...... te hacen sentir muy comodo tanto las '
 'mozas como el dueño.\r\n'
 'Los platos son riquisimos.... el menu degustación ( de 7 pasos) es '
 'exquisito... repito que soy un novato de la cultura gastronomica pero   '
 'platos, atencion y precios.... un 10 a todos.',
 'El restaurante se quedo en el tiempo. El ambiente vintage, aunque limpio. '
 'Los mozos, buenos veteranos como los que se solian encontrar. La comida '
 'maso: lomo marsala estaba bien (dos tajadas con papines, $50,) costilla '
 'riojana ($60, dos costillas con bastante nervio y grasa, el jamon crudo muy '
 'salado, los pimientos sin gusto, y las arvejas parecian de lata, el huevo '
 'frito un poco seco.) La panera no mer

## Tokenización

Transforme cada elemento de *comentarios* en una lista de palabras utilizando las funciones de tokenización de oraciones de nltk. Almacene el resultado en una variable llamada *corpus_tokens*. Note que *corpus_tokens* es una lista donde cada elemento es a su vez una lista con los *tokens* del comentario correspondiente.

In [23]:
#Usamos primero sent_tokenize para partir los comentarios en oraciones y luego word_tokenize para partir esas oraciones en palabras

corpus_tokens = [[word for sent in nltk.tokenize.sent_tokenize(comment, language='spanish') for word in nltk.tokenize.word_tokenize(sent)] for comment in comentarios]

Muestre los dos últimos elementos de *corpus_tokens*.

In [24]:
pprint(corpus_tokens[:2])

[['Algo',
  'diferente',
  'sobre',
  'todo',
  'por',
  'la',
  'cordialidad',
  'y',
  'el',
  'servicio',
  '.',
  'Es',
  'en',
  'una',
  'casa',
  'antigua',
  ',',
  'dentro',
  'de',
  'una',
  'biblioteca',
  'publica',
  ',',
  'la',
  'comida',
  'excelentemente',
  'casera',
  ',',
  'y',
  'la',
  'atencion',
  'buenisima',
  '.'],
 ['soy',
  'un',
  'neófito',
  'en',
  'la',
  'cultura',
  'gastronómica',
  'de',
  'buen',
  'nivel',
  'y',
  'aún',
  'así',
  'el',
  'trato',
  'y',
  'la',
  'amabilidad',
  'son',
  'destacables',
  '...',
  '...',
  'te',
  'hacen',
  'sentir',
  'muy',
  'comodo',
  'tanto',
  'las',
  'mozas',
  'como',
  'el',
  'dueño',
  '.',
  'Los',
  'platos',
  'son',
  'riquisimos',
  '...',
  '.',
  'el',
  'menu',
  'degustación',
  '(',
  'de',
  '7',
  'pasos',
  ')',
  'es',
  'exquisito',
  '...',
  'repito',
  'que',
  'soy',
  'un',
  'novato',
  'de',
  'la',
  'cultura',
  'gastronomica',
  'pero',
  'platos',
  ',',
  'atencion',


Realice otra tokenización de los *comentarios* utilizando expresiones regulares y el módulo re de Python. En esta oportunidad cumpla con las siguientes restricciones:

- Reconocer tokens de por lo menos 3 caracteres de largo.
- No reconocer símbolos de puntuación.
- No reconocer tokens únicamente numéricos.

Almacene el resultado en una variable llamada *corpus_tokens2*.

In [6]:
corpus_tokens2 = [re.findall(r'\b(\d*[^\W\d]+\d*[^\W\d]*\d*)\b', comment) for comment in comentarios]

Despliegue los dos últimos elementos de *corpus_tokens2*.

In [15]:
pprint(comentarios[:2])
pprint(corpus_tokens2[:2])

['Algo diferente sobre todo por la cordialidad y el servicio. Es en una casa '
 'antigua, dentro de una biblioteca publica, la comida excelentemente '
 'casera,   y la atencion buenisima.',
 'soy un neófito en la cultura gastronómica de buen nivel y aún así el trato y '
 'la amabilidad son destacables...... te hacen sentir muy comodo tanto las '
 'mozas como el dueño.\r\n'
 'Los platos son riquisimos.... el menu degustación ( de 7 pasos) es '
 'exquisito... repito que soy un novato de la cultura gastronomica pero   '
 'platos, atencion y precios.... un 10 a todos.']
[['Algo',
  'diferente',
  'sobre',
  'todo',
  'por',
  'la',
  'cordialidad',
  'y',
  'el',
  'servicio',
  'Es',
  'en',
  'una',
  'casa',
  'antigua',
  'dentro',
  'de',
  'una',
  'biblioteca',
  'publica',
  'la',
  'comida',
  'excelentemente',
  'casera',
  'y',
  'la',
  'atencion',
  'buenisima'],
 ['soy',
  'un',
  'neófito',
  'en',
  'la',
  'cultura',
  'gastronómica',
  'de',
  'buen',
  'nivel',
  'y',
  

# Bag of Words

Utilice la clase CountVectorizer del módulo sklearn.feature_extraction.text para instanciar un objeto que transforme el corpus en un diccionario de las palabras más representativas de las oraciones del corpus. Investigue cada uno de los parámetros que acepta su constructor y configurelos de forma que considere adecuada. Justifique sus decisiones.

In [8]:
transf = sklearn.feature_extraction.text.CountVectorizer(max_df=0.95, min_df=100)

Entrene la transformación e imprima las palabras resultantes en el diccionario. Imprima además la cantidad de palabras obtenidas en el diccionario.

In [9]:
transf.fit(comentarios)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.95, max_features=None, min_df=100,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [10]:
diccionario = transf.get_feature_names()
print(diccionario)
print("El diccionario tiene " + str(len(diccionario)) + " palabras.")

['00', '10', '100', '11', '12', '14', '15', '150', '20', '200', '21', '22', '23', '25', '30', '35', '40', '45', '50', '60', '70', '80', '90', 'abajo', 'abierto', 'absolutamente', 'absoluto', 'abuela', 'abundante', 'abundantes', 'aca', 'accesible', 'accesibles', 'aceite', 'aceitosas', 'aceitunas', 'aceptable', 'acerca', 'achuras', 'acogedor', 'acompaña', 'acompañada', 'acompañado', 'acompañamiento', 'acompañar', 'acondicionado', 'acorde', 'acordes', 'acotada', 'acuerdo', 'acá', 'adecuada', 'adecuado', 'ademas', 'además', 'adentro', 'afuera', 'agradable', 'agradables', 'agua', 'aguas', 'ah', 'ahi', 'ahora', 'ahumado', 'ahí', 'aire', 'aires', 'ajo', 'al', 'alcohol', 'algo', 'alguien', 'algun', 'alguna', 'algunas', 'algunos', 'algún', 'alla', 'alli', 'allá', 'allí', 'almorzar', 'almuerzo', 'alrededor', 'alta', 'altamente', 'alternativa', 'alto', 'altos', 'altura', 'amabilidad', 'amable', 'amables', 'amantes', 'ambas', 'ambientacion', 'ambientación', 'ambientado', 'ambiente', 'ambos', 'amen

Transforme todos los *comentarios* en vectores utilizando la transformación creada. Guarde las transformaciones en una lista llamada 'vectores':

In [11]:
vectores = transf.transform(comentarios)

## Similitud

Escriba una función calcular_similitud(vect1, vect2) que dados dos vectores, devuelva un número en el rango [0, 1] donde 1 significa que los vectores son idénticos y 0 que son completamente diferentes. Puede utilizar medidas como la implementada en la función cosine del módulo scipy.spatial.distance u otra que considere adecuada.

Vamos a definir los siguientes dos diccionarios:

In [12]:
similar = {}
similitud = {}

El primer diccionario guardará para cada oración, el índice de la oración más similar a ella. Esto es, similar[i] = j indica que la j-esima oración, es la oración más similar a la i-esima oración.
El segundo diccionario guardará el valor de similitud correspondiente. Si similiar[i] = j, entonces similitud[i] = calcular_similitud(oracion[i], oracion[j]).
Implemente una fución llamada calcular_similitudes(vectores, similar, similitud) que dada la lista de vectores, construya los diccionarios similar y similitud:

Implemente una función llamada *imprimir_similares* que imprima los primeros N pares de comentarios más similares del corpus. Utilice un valor que considere adecuado para N (ej. N=50).

Ejecute las funciones calcular_similitudes e imprimir_similares. Utilice una porción del corpus que considere adecuada para que ejecute en un tiempo razonable (ej. los primeros 1000 comentarios). Despliegue los resultados y el tiempo transcurrido. Si lo desea puede incluir resultados en un bloque de texto al considerar una porción mayor. De ser así incluya la cantidad de elementos considerados y el tiempo transcurrido. 

Analice los resultados obtenidos. ¿Que observa? ¿En qué opina que se podría mejorar?

## Bag of Words (con stemming)

Utilice el parámetro 'tokenizer' de la clase CountVectorizer y conjuntamente la clase SpanishStemmer del módulo nltk.stem.snowball. Repita los experimentos realizados en la parte anterior utilizando el stemmer. Realice las comparaciones que considere pertinentes.

Al igual que en la parte anterior muestre los resultados para una porción del corpus y si lo desea incluya los resultados para una porción mayor. Despliegue el tiempo transcurrido.

Justifique las decisiones tomadas y realice los comentarios que considere adecuados. Analice los resultados obtenidos. Compare los resultados con los obtenidos anteriormente. Comente los problemas que detecte en este enfoque.